<a href="https://colab.research.google.com/github/NielsRogge/Transformers-Tutorials/blob/master/VisionTransformer/Fine_tuning_the_Vision_Transformer_on_CIFAR_10_with_the_%F0%9F%A4%97_Trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This code is directly adapted from the tutorial provided here: https://github.com/NielsRogge/Transformers-Tutorials/tree/master/VisionTransformer




In [ ]:
!pip install accelerate==0.15.0
!pip install transformers==4.28.1
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.5/191.5 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.1.1
    Uninstalling accelerate-1.1.1:
      Successfully uninstalled accelerate-1.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
peft 0.13.2 requires accelerate>=0.21.0, but you have accelerate 0.15.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.0/110.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 120.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
  Attempting uninstall: transformers
    Found existing installation: tran

In [ ]:
from datasets import load_dataset, Dataset, Features, Image, ClassLabel

dataset = load_dataset("sled-umich/Action-Effect", trust_remote_code=True)
data_as_list = [
    {"img": img, "label": block["verb noun"]}
    for block in dataset["ActionEffect"]
    for img in block["positive_image_list"]
] + [
    {"img": img, "label": "no effect"}
    for block in dataset["ActionEffect"]
    for img in block["negative_image_list"]
]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.78k [00:00<?, ?B/s]

Action-Effect.py:   0%|          | 0.00/5.07k [00:00<?, ?B/s]

action_effect_info.json:   0%|          | 0.00/195k [00:00<?, ?B/s]

action_effect_image_rs.zip:   0%|          | 0.00/408M [00:00<?, ?B/s]

Generating ActionEffect split: 0 examples [00:00, ? examples/s]

In [ ]:
label_list = [
    block['verb noun']
    for block in dataset['ActionEffect']
]
label_list.append('no effect')

fts= Features({
    "img": Image(),
    "label": ClassLabel(names=unique_labels),
})


dataset = Dataset.from_list(data_as_list, features=fts)
dataset = dataset.shuffle(seed=595)

first_split = dataset.train_test_split(test_size=0.6, seed=595)

second_split = first_split["train"]
test_ds = first_split["test"]

train_val_split = second_split.train_test_split(test_size=0.25, seed=595)

train_ds = train_val_split["train"]
val_ds = train_val_split["test"]

print(len(train_dataset), len(val_dataset), len(test_dataset))


Train size: 1248
Validation size: 417
Test size: 2498


In [ ]:
train_ds.features

{'img': Image(mode=None, decode=True, id=None),
 'label': ClassLabel(names=['arrange chairs', 'arrange flowers', 'bake potato', 'beat eggs', 'bend knee', 'bend tree', 'bind hair', 'bite apple', 'block door', 'block window', 'boil egg', 'boil potato', 'break bowl', 'break cup', 'break door', 'break egg', 'break glass', 'break window', 'burn book', 'burn paper', 'burn tree', 'burn wood', 'burst balloon', 'burst door', 'carry bag', 'carry book', 'carry umbrella', 'chop carrot', 'chop meat', 'chop onion', 'chop tree', 'chop wood', 'close book', 'close cabinet', 'close door', 'close drawer', 'close window', 'coil rope', 'cook egg', 'cook meat', 'cook onion', 'cook potato', 'crack bottle', 'crack egg', 'crack glass', 'crack window', 'crash car', 'crop hair', 'cut apple', 'cut meat', 'cut onion', 'cut potato', 'cut tree', 'cut wood', 'fasten door', 'fasten window', 'fold paper', 'fry egg', 'fry meat', 'fry potato', 'grate carrot', 'grate potato', 'grind meat', 'hang bag', 'hang shirt', 'ignit

In [ ]:
train_ds[0]['label']

125

In [ ]:
id2label = {id:label for id, label in enumerate(train_ds.features['label'].names)}
label2id = {label:id for id,label in id2label.items()}
id2label

{0: 'arrange chairs',
 1: 'arrange flowers',
 2: 'bake potato',
 3: 'beat eggs',
 4: 'bend knee',
 5: 'bend tree',
 6: 'bind hair',
 7: 'bite apple',
 8: 'block door',
 9: 'block window',
 10: 'boil egg',
 11: 'boil potato',
 12: 'break bowl',
 13: 'break cup',
 14: 'break door',
 15: 'break egg',
 16: 'break glass',
 17: 'break window',
 18: 'burn book',
 19: 'burn paper',
 20: 'burn tree',
 21: 'burn wood',
 22: 'burst balloon',
 23: 'burst door',
 24: 'carry bag',
 25: 'carry book',
 26: 'carry umbrella',
 27: 'chop carrot',
 28: 'chop meat',
 29: 'chop onion',
 30: 'chop tree',
 31: 'chop wood',
 32: 'close book',
 33: 'close cabinet',
 34: 'close door',
 35: 'close drawer',
 36: 'close window',
 37: 'coil rope',
 38: 'cook egg',
 39: 'cook meat',
 40: 'cook onion',
 41: 'cook potato',
 42: 'crack bottle',
 43: 'crack egg',
 44: 'crack glass',
 45: 'crack window',
 46: 'crash car',
 47: 'crop hair',
 48: 'cut apple',
 49: 'cut meat',
 50: 'cut onion',
 51: 'cut potato',
 52: 'cut t

In [ ]:
id2label[train_ds[0]['label']]

'strip tree'

In [ ]:
from transformers import ViTImageProcessor

processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

In [ ]:
from torchvision.transforms import (CenterCrop,
                                    Compose,
                                    Normalize,
                                    RandomHorizontalFlip,
                                    RandomResizedCrop,
                                    Resize,
                                    ToTensor)

image_mean, image_std = processor.image_mean, processor.image_std
size = processor.size["height"]

normalize = Normalize(mean=image_mean, std=image_std)
_train_transforms = Compose(
        [
            RandomResizedCrop(size),
            RandomHorizontalFlip(),
            ToTensor(),
            normalize,
        ]
    )

_val_transforms = Compose(
        [
            Resize(size),
            CenterCrop(size),
            ToTensor(),
            normalize,
        ]
    )

def train_transforms(examples):
    examples['pixel_values'] = [_train_transforms(image.convert("RGB")) for image in examples['img']]
    return examples

def val_transforms(examples):
    examples['pixel_values'] = [_val_transforms(image.convert("RGB")) for image in examples['img']]
    return examples

In [ ]:
# Set the transforms
train_ds.set_transform(train_transforms)
val_ds.set_transform(val_transforms)
test_ds.set_transform(val_transforms)

In [ ]:
train_ds[:2]

{'img': [<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=186x270>,
  <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=448x448>],
 'label': [125, 140],
 'pixel_values': [tensor([[[ 0.9843,  0.9843,  0.9843,  ...,  0.9843,  0.9843,  0.9843],
           [ 0.9843,  0.9843,  0.9843,  ...,  0.9843,  0.9843,  0.9843],
           [ 0.9843,  0.9843,  0.9843,  ...,  0.9843,  0.9843,  0.9843],
           ...,
           [-0.1451, -0.1686, -0.1843,  ...,  0.9843,  0.9843,  0.9843],
           [-0.1294, -0.1608, -0.1843,  ...,  0.9843,  0.9843,  0.9843],
           [-0.1216, -0.1608, -0.1765,  ...,  0.9843,  0.9843,  0.9843]],
  
          [[ 0.9765,  0.9765,  0.9765,  ...,  0.9765,  0.9765,  0.9765],
           [ 0.9765,  0.9765,  0.9765,  ...,  0.9765,  0.9765,  0.9765],
           [ 0.9765,  0.9765,  0.9765,  ...,  0.9765,  0.9765,  0.9765],
           ...,
           [-0.1373, -0.1765, -0.2000,  ...,  0.9765,  0.9765,  0.9765],
           [-0.1216, -0.1686, -0.2000,  ...,  0.9765, 

In [ ]:
from torch.utils.data import DataLoader
import torch

def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

train_dataloader = DataLoader(train_ds, collate_fn=collate_fn, batch_size=4)

In [ ]:
batch = next(iter(train_dataloader))
for k,v in batch.items():
  if isinstance(v, torch.Tensor):
    print(k, v.shape)

pixel_values torch.Size([4, 3, 224, 224])
labels torch.Size([4])


In [ ]:
from transformers import ViTForImageClassification

model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224-in21k',
                                                  id2label=id2label,
                                                  label2id=label2id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments, Trainer

metric_name = "accuracy"

args = TrainingArguments(
    f"test-actioneffect",
    save_strategy="epoch",
    evaluation_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    weight_decay=0.005,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    logging_dir='logs',
    remove_unused_columns=False,
)

In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    top1_pred = np.argmax(predictions, axis=1)

    top1_accuracy = accuracy_score(top1_pred, labels)

    top5_pred = np.argsort(predictions, axis=1)[:, -5:]
    top5_accuracy = np.mean([label in top5_pred[i] for i, label in enumerate(labels)])

    top20_pred = np.argsort(predictions, axis=1)[:, -20:]
    top20_accuracy = np.mean([label in top20_pred[i] for i, label in enumerate(labels)])

    return {
        'accuracy': top1_accuracy,
        'top_5_accuracy': top5_accuracy,
        'top_20_accuracy': top20_accuracy
    }

In [ ]:
import torch

trainer = Trainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    tokenizer=processor,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Top 5 Accuracy,Top 20 Accuracy
1,No log,4.226665,0.517986,0.741007,0.817746
2,No log,4.009495,0.498801,0.731415,0.822542
3,No log,4.536773,0.505995,0.709832,0.803357
4,No log,4.020469,0.505995,0.745803,0.822542
5,No log,4.164316,0.503597,0.733813,0.822542


/usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2220: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(best_model_path, map_location="cp

TrainOutput(global_step=195, training_loss=0.0671570313282502, metrics={'train_runtime': 136.99, 'train_samples_per_second': 45.551, 'train_steps_per_second': 1.423, 'total_flos': 4.841524281065472e+17, 'train_loss': 0.0671570313282502, 'epoch': 5.0})

In [ ]:
outputs = trainer.predict(test_ds)

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [ ]:
print(outputs.metrics)

{'test_loss': 3.8392975330352783, 'test_accuracy': 0.5372297838270617, 'test_top_5_accuracy': 0.7345876701361089, 'test_top_20_accuracy': 0.8306645316253003, 'test_runtime': 34.4621, 'test_samples_per_second': 72.485, 'test_steps_per_second': 18.136}
